<a href="https://colab.research.google.com/github/ozawa-nami/test/blob/main/%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E6%B1%BA%E5%AE%9A%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

テストテスト

In [ ]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


!pip install selenium
!pip install webdriver_manager pyvirtualdisplay
!apt-get update
!apt-get install -y xvfb wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 仮想ディスプレイの起動
display = Display(visible=0, size=(1024, 768))
display.start()

# Chrome のオプションとサービスの設定
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())
#s = Service(ChromeDriverManager("116.0.5845.140").install())


# Chrome ブラウザの起動
driver = webdriver.Chrome(service=s, options=options)

In [ ]:
#1行ずつ書き出し


# 必要なライブラリのインストール
!pip install validators

# 必要なライブラリのインポート
import gspread
import google.auth
import time
from google.colab import auth
from google_auth_oauthlib.flow import InstalledAppFlow
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from oauth2client.client import GoogleCredentials
from soupsieve import SelectorSyntaxError
import validators  # URL検証用のライブラリ



# Googleドライブへの認証を行います
auth.authenticate_user()

# 認証情報を保存するための変数を初期化します
creds = None

# Google Colab環境での認証
if 'google.colab' in str(get_ipython()):
    creds, _ = google.auth.default()
else:
    # ローカル環境の場合、クライアントの秘密ファイルを使用して認証を行います
    # 以下の行の`scraping-384605-1e7cd0ea5328.json`は、Google Cloud Consoleから取得したクライアントの秘密ファイル名に置き換えてください
    flow = InstalledAppFlow.from_client_secrets_file('scraping-384605-1e7cd0ea5328.json', scopes)
    creds = flow.run_local_server(port=0)

# gspreadライブラリに認証情報を設定し、Googleスプレッドシートにアクセスできるようにします
gc = gspread.authorize(creds)

# スプレッドシートから全行のデータを取得

# スプレッドシートのURL
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1NwFp7_HG64TFHFE7VfppyqDgnMUZ6UI8DYJzR7ETOvc/edit#gid=807235530"


worksheet = gc.open_by_url(spreadsheet_url).worksheet("はたらいく")


# スプレッドシートから全行のデータを取得
all_data = worksheet.get_all_values()
header = all_data[0]  # ヘッダー行
data_rows = all_data[4:]  # 5行目からのデータ行（0からカウントするため、4を指定）


# ChromeのWebドライバ設定
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
s = Service(ChromeDriverManager().install()) #　webdriverとChromeのバージョン合わない時は、直接指定
#ChromeDriverManager("116.0.5845.140").install()
driver = webdriver.Chrome(service=s, options=options)

# 各行に対してスクレイピングを実行
for i, row in enumerate(data_rows, start=5):  # 5行目からカウント開始
    url = row[0]  # A列の値
    selector = row[1]  # B列の値

    if not url or not validators.url(url):
        print(f"Invalid URL at row {i}: {url}")
        continue  # URLが無効な場合はスキップ

    # URLにアクセス
    try:
        driver.get(url)
        # ページの全要素がロードされるまで待機
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    except TimeoutException:
        print(f"Timeout while waiting for page to load: {url}")
        continue  # タイムアウト時に次のループに進む

    # ページのHTMLを取得
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # セレクタに基づいてデータを抽出
    try:
        element = soup.select_one(selector)
        if element:
            result = element.get_text(strip=True)
        else:
            result = 'Not Found'
    except SelectorSyntaxError as e:
        print(f"Invalid selector '{selector}' at row {i}: {str(e)}")
        result = 'Error'

    # スクレイピングの結果をスプレッドシートに書き込む
    result_cell = gspread.models.Cell(i, 3, result)  # 書き込むセルの行番号をiで指定
    worksheet.update_cell(result_cell.row, result_cell.col, result_cell.value)


# all_data = worksheet.get_all_values()
# header = all_data[0]  # ヘッダー行
# data_rows = all_data[4:]  # データ行（3行目から開始）

# # ChromeのWebドライバ設定

# options = webdriver.ChromeOptions()
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--headless')

# s = Service(ChromeDriverManager().install())   #　webdriverとChromeのバージョン合わない時は、直接指定
# #ChromeDriverManager("116.0.5845.140").install()
# driver = webdriver.Chrome(service=s, options=options)

# # 各行に対してスクレイピングを実行
# for i, row in enumerate(data_rows):
#     url = row[0]  # A列の値
#     selector = row[1]  # B列の値

#     if not url or not validators.url(url):
#         print(f"Invalid URL at row {i+4}: {url}")
#         continue  # URLが無効な場合はスキップ

#     # URLにアクセス
#     try:
#         driver.get(url)

#         # ページの全要素がロードされるまで待機
#         WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
#     except TimeoutException:
#         print(f"Timeout while waiting for page to load: {url}")
#         continue  # タイムアウト時に次のループに進む



#     # ページのHTMLを取得
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')

#     # セレクタに基づいてデータを抽出
#     try:
#         element = soup.select_one(selector)
#         if element:
#             result = element.get_text(strip=True)
#         else:
#             result = 'Not Found'
#     except SelectorSyntaxError as e:
#         print(f"Invalid selector '{selector}' at row {i+4}: {str(e)}")
#         result = 'Error'

#     # スクレイピングの結果をスプレッドシートに書き込む
#     result_cell = gspread.models.Cell(i + 4, 23, result)
#     worksheet.update_cell(result_cell.row, result_cell.col, result_cell.value)


# ウェブドライバを閉じる
driver.quit()


In [ ]:

#まとめて書き出す

# 必要なライブラリのインポート
import gspread  # Google Sheetsとの連携に使用します
import google.auth  # 認証情報の取得に使用します
import time  # ウェイト処理に使用します
from google.colab import auth  # Google Colab環境での認証に使用します
from google_auth_oauthlib.flow import InstalledAppFlow  # ローカル環境での認証に使用します
from selenium import webdriver  # ウェブスクレイピングに使用します
from selenium.webdriver.chrome.service import Service  # Chromeドライバの設定に使用します
from webdriver_manager.chrome import ChromeDriverManager  # Chromeドライバの自動インストールに使用します
from bs4 import BeautifulSoup  # HTMLの解析に使用します
from selenium.webdriver.support.ui import WebDriverWait  # 要素の待機に使用します
from selenium.webdriver.support import expected_conditions as EC  # 条件の指定に使用します
from oauth2client.client import GoogleCredentials  # Google APIの認証に使用します
from soupsieve import SelectorSyntaxError


# Googleドライブへの認証を行います
auth.authenticate_user()

# 認証情報を保存するための変数を初期化します
creds = None

# Google Colab環境での認証
if 'google.colab' in str(get_ipython()):
    creds, _ = google.auth.default()
else:
    # ローカル環境の場合、クライアントの秘密ファイルを使用して認証を行います
    # 以下の行の`scraping-384605-1e7cd0ea5328.json`は、Google Cloud Consoleから取得したクライアントの秘密ファイル名に置き換えてください
    flow = InstalledAppFlow.from_client_secrets_file('scraping-384605-1e7cd0ea5328.json', scopes)
    creds = flow.run_local_server(port=0)

# gspreadライブラリに認証情報を設定し、Googleスプレッドシートにアクセスできるようにします
gc = gspread.authorize(creds)

# スクレイピング対象のスプレッドシートのURLとシート名を定義します
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1NwFp7_HG64TFHFE7VfppyqDgnMUZ6UI8DYJzR7ETOvc/edit#gid=348990538"
worksheet_name = "テスト"

# スプレッドシートを開き、シートを指定します
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.worksheet(worksheet_name)

# スプレッドシートからURLとセレクタの情報を読み込みます
url_list = worksheet.range("A3:A438")
selector_list = worksheet.range("B3:B438")

# スクレイピング結果を格納するためのリストを初期化します
results = []

# ChromeのWebドライバを設定し、ブラウザを起動します
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())   #　webdriverとChromeのバージョン合わない時は、直接指定
#ChromeDriverManager("116.0.5845.140").install()
driver = webdriver.Chrome(service=s, options=options)



# 各URLに対してスクレイピングを行います
for i, url_cell in enumerate(url_list):
    url = url_cell.value
    if not url:
        continue  # URLがない場合はスキップする

    # ウェブドライバに指定したURLを開かせます
    try:
        driver.get(url)
    except Exception as e:
        print(f"URL '{url}' の読み込み中にエラーが発生しました: {str(e)}")
        continue

    # ページ上の全ての要素が読み込まれるまで待機します（最大で5秒）
    WebDriverWait(driver, 1).until(EC.presence_of_all_elements_located)

    # ページのHTMLを取得します
    html = driver.page_source.encode('utf-8')

    # BeautifulSoupでHTMLをパースします
    soup = BeautifulSoup(html, 'html.parser')

    # セレクタを取得します
    selector = selector_list[i].value
    if not selector:
        # セレクタが存在しない場合、結果のリストに空文字列を追加します
        results.append(("", url_cell))
        continue



    # セレクタに対応するテキストを取得します
    try:
        text = soup.select_one(selector)
    except SelectorSyntaxError:
        print(f"セレクタ '{selector}' が不適切です。スキップします。")
        continue

    if text:
        try:
            # テキストを浮動小数点数に変換します
            float_val = float(text.text.replace(",", ""))
            results.append((float_val, url_cell))
        except ValueError:
            # テキストが数値に変換できない場合、そのまま結果のリストに追加します
            results.append((text.text, url_cell))
    else:
        # テキストが存在しない場合、結果のリストに空文字列を追加します
        results.append(("", url_cell))


# スクレイピング結果を一括で書き込みます
cells_to_update = []
for result, cell in results:
    cells_to_update.append(gspread.models.Cell(cell.row, 7, result))  # 列に結果を書き込みます（列の列番号を表します）
worksheet.update_cells(cells_to_update)


# ウェブドライバを閉じます
driver.quit()
